In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Optional, Union, Tuple, List

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import OneCycleLR

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")

import optuna
from optuna.visualization import (
    plot_edf
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances

    , plot_slice
)

# local modules
import sys
sys.path.append("../src")
from preproc import preprocess_data

In [ ]:
from lightning.pytorch import seed_everything
seed_everything(2112, workers=True);

***
### load and preprocess data

In [ ]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

In [ ]:
# Call the function
df_train, df_test, numerical_cols, categorical_cols = preprocess_data(
    df_train,
    df_test,
    scale_utility=True
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

In [ ]:
plt.hist(df_train['utility_agent1_scaled'], bins=100)
plt.title('Distribution of Target Variable')
plt.xlabel('Utility Agent 1')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_train[numerical_cols] = df_train[numerical_cols].astype(np.float32)
df_train[categorical_cols] = df_train[categorical_cols].astype(np.int32)

cat_input_dims = df_train[categorical_cols].nunique(axis=0).values.tolist()
print(cat_input_dims)

In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical columns of df_train
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols])

# Print a message to confirm the scaling
print("Numerical columns have been scaled using StandardScaler.")


***
### train model


In [ ]:
class MLP(pl.LightningModule):

    def __init__(self, 
            num_input_dim: int,
            cat_input_dims: list[int],
            output_dim: int,
            layers: str,
            dropout: float,
            learning_rate: float = 1e-3,
            weight_decay: float = 1e-5,
            initialization: str = 'kaiming_uniform',
            embedding_dim: Optional[List[int]] = None,
        ):
        super().__init__()
        self.save_hyperparameters()

        # Initialize embedding dimensions if not provided
        if embedding_dim is None:
            # Rule of thumb: min(50, num_unique // 2 + 1) for each categorical feature
            embedding_dim = [min(50, int(1 + np.ceil(np.sqrt(dim)))) for dim in cat_input_dims]

        elif len(embedding_dim) != len(cat_input_dims):
            raise ValueError("Length of embedding_dim must match number of categorical features.")

        self.embedding_dim = embedding_dim

        # Create embedding layers
        self.create_embeddings(cat_input_dims, embedding_dim)

        # Create backbone layers
        self.create_backbone(num_input_dim, layers)

        # Create head layers
        self.create_head(output_dim)

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.initialization = initialization

        self._init_weights()

        # Initialize lists to store validation outputs
        self.validation_targets = []
        self.validation_predictions = []

    def create_embeddings(self, cat_input_dims: list[int], embedding_dim: list[int]):
        self.embeddings = nn.ModuleList(
            [nn.Embedding(dim, emb_dim) for dim, emb_dim in zip(cat_input_dims, embedding_dim)]
        )

    def create_backbone(self, num_input_dim: int, layers: str):
        # Calculate total input dimension after embeddings
        total_embedding_dim = sum(self.embedding_dim)
        total_input_dim = num_input_dim + total_embedding_dim

        # Parse layers string
        layer_sizes = [int(size) for size in layers.split('-')]

        # Create backbone network layers
        backbone_layers = []
        prev_size = total_input_dim
        for size in layer_sizes:
            backbone_layers.extend([
                nn.BatchNorm1d(prev_size),
                nn.Linear(prev_size, size),
                nn.ReLU(),
                nn.Dropout(self.hparams.dropout),
            ])
            prev_size = size
        self.backbone = nn.Sequential(*backbone_layers)
        self.backbone_output_size = prev_size

    def create_head(self, output_dim: int):
        # Output layer
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.backbone_output_size),
            nn.Linear(self.backbone_output_size, output_dim)
        )

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                if any(module is m for m in self.head.modules()):
                    nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('tanh'))
                else:
                    if self.initialization == 'kaiming_uniform':
                        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'kaiming_normal':
                        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'xavier_uniform':
                        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
                    elif self.initialization == 'xavier_normal':
                        nn.init.xavier_normal_(module.weight, gain=nn.init.calculate_gain('relu'))
                    else:
                        raise ValueError(f"Unsupported initialization method: {self.initialization}")
                
                # Initialize bias to small values
                if module.bias is not None:
                    nn.init.uniform_(module.bias, -0.1, 0.1)

    def forward(self, x_num, x_cat):
        # Process categorical variables
        embedded = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        
        # Concatenate numerical and embedded categorical features
        x = torch.cat([x_num, embedded], dim=1)
        
        # Pass through backbone
        x = self.backbone(x)
        
        # Pass through head
        x = self.head(x)
        x = nn.functional.hardtanh(x)

        return x.squeeze(-1)

    def training_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss, prog_bar=True)
        # Store targets and predictions for later use
        self.validation_targets.append(y)
        self.validation_predictions.append(y_hat)
        return loss
    
    def predict_step(self, batch, batch_idx):
        if len(batch) == 2:
            x_num, x_cat = batch
        elif len(batch) == 3:
            x_num, x_cat, _ = batch
        y_hat = self(x_num, x_cat)
        return y_hat

    def on_validation_epoch_end(self):
        # Concatenate all targets and predictions
        y = torch.cat(self.validation_targets)
        y_hat = torch.cat(self.validation_predictions)
        rmse = torch.sqrt(F.mse_loss(y_hat, y))
        self.log('val_rmse', rmse, prog_bar=True)
        # Clear the lists for next epoch
        self.validation_targets.clear()
        self.validation_predictions.clear()
                
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.learning_rate, 
            weight_decay=self.weight_decay,
        )
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.learning_rate,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=0.2,
            anneal_strategy='cos',
            cycle_momentum=True,
            base_momentum=0.85,
            max_momentum=0.95,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [ ]:
# Define the number of folds for cross-validation
num_folds = 5

# Define the column for stratified or group k-fold
groups_col = 'GameRulesetName'
gkf = GroupKFold(n_splits=num_folds)
split_list = list(gkf.split(df_train, groups=df_train[groups_col]))

early_stop_callback = EarlyStopping(
    monitor='val_rmse',
    patience=10,
    mode='min',
    verbose=False
)

def train_and_score(
        batch_size,
        dropout,
        input_layer_size,
        n_layers,
        learning_rate,
        weight_decay,
        initialization,
        pct_start,
    ):
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        
        # Split the data
        train, valid = df_train.iloc[train_index], df_train.iloc[val_index]

        train_dataset = TensorDataset(
            torch.tensor(train[numerical_cols].values, dtype=torch.float32),
            torch.tensor(train[categorical_cols].values, dtype=torch.int32),
            torch.tensor(train['utility_agent1'].values, dtype=torch.float32)
        )
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=8,
            persistent_workers=True
        )

        valid_dataset = TensorDataset(
            torch.tensor(valid[numerical_cols].values, dtype=torch.float32),
            torch.tensor(valid[categorical_cols].values, dtype=torch.int32),
            torch.tensor(valid['utility_agent1'].values, dtype=torch.float32)
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=8,
            persistent_workers=True
        )

        # Construct the layers string based on input_layer_size and n_layers
        layers = "-".join([str(input_layer_size // (2 ** i)) for i in range(n_layers)])

        model = MLP(
            num_input_dim=len(numerical_cols),
            cat_input_dims=cat_input_dims,
            output_dim=1,
            layers=layers,
            dropout=dropout,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            initialization=initialization,
            pct_start=pct_start,
        )
        trainer = pl.Trainer(
            min_epochs=20,
            max_epochs=100, 
            deterministic=True,
            accelerator="mps",
            enable_progress_bar=False,
            enable_model_summary=False,
            callbacks=[
                early_stop_callback, 
                ModelCheckpoint(monitor='val_rmse', mode='min', save_top_k=1),
            ],
        );
        trainer.fit(
            model, 
            train_loader,
            valid_loader,
        );

        # Load the best model
        best_model_path = trainer.checkpoint_callback.best_model_path
        model = MLP.load_from_checkpoint(best_model_path)

        # Predict on validation set using trainer.predict with the prediction DataLoader
        predictions = trainer.predict(model, dataloaders=valid_loader)
        y_pred = torch.cat(predictions).squeeze().cpu().numpy()
         
        # Compute RMSE on scaled values
        y_valid = valid['utility_agent1'].values
        rmse = np.sqrt(np.mean((y_pred - y_valid) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [ ]:
def objective(trial):
    # Suggest hyperparameters to tune
    batch_size = trial.suggest_int('batch_size', 128, 512, step=64)
    dropout = trial.suggest_float('dropout', 0.0, 0.3, step=0.05)
    input_layer_size = trial.suggest_int('input_layer_size', 512, 1024, step=64)
    n_layers = trial.suggest_int('n_layers', 2, 4)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-4, 1e-2, log=True)
    pct_start = trial.suggest_float('pct_start', 0.05, 0.2, step=0.05)
    div_factor = trial.suggest_float('div_factor', 1.0, 100.0, step=1.0)
    final_div_factor = trial.suggest_float('final_div_factor', 1e0, 1e4, log=True)

    # Train and evaluate the model with the suggested hyperparameters
    oof_score = train_and_score(
        batch_size=batch_size,
        dropout=dropout,
        input_layer_size=input_layer_size,
        n_layers=n_layers,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        initialization="kaiming_uniform",
        pct_start=pct_start,
        div_factor=div_factor,
        final_div_factor=final_div_factor,
    )

    return oof_score


In [ ]:
do_optimize = True
timeout = 3600 * 24

study = optuna.create_study(
    study_name="mlp",
    direction='minimize',
    storage='sqlite:///mlp.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=1000, 
        timeout=timeout,
        n_jobs=1, 
        gc_after_trial=True,
    ) 

In [ ]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_edf(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
best_params = dict(study.best_params)
best_params

***